PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
import keras

In [2]:
data=pd.read_csv('Forestfires.csv')

In [3]:
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [4]:
from sklearn.preprocessing import LabelEncoder
l_en=LabelEncoder()
data['month']=l_en.fit_transform(data['month'])
data['day']=l_en.fit_transform(data['day'])

In [5]:
data1=data.iloc[:,0:11]
data1

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,10,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,10,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
512,1,3,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,1,3,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,1,3,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,2,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [6]:
x=data1.drop('area',axis=1)
y=data1['area']

In [7]:
from sklearn.preprocessing import StandardScaler
a=StandardScaler()
x_standardized=a.fit_transform(x)

In [8]:
pd.DataFrame(x_standardized).describe()

,0,1,2,3,4,5,6,7,8,9
count,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02,5.170000e+02
mean,-4.874674e-17,5.110891e-17,-1.754024e-15,3.070830e-16,7.387171e-17,-3.865380e-17,2.005703e-16,3.362881e-16,-2.676776e-16,-2.841054e-16
std,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00,1.000969e+00
min,-1.317959e+00,-1.423121e+00,-1.304582e+01,-1.715608e+00,-2.179108e+00,-1.980578e+00,-2.876943e+00,-1.796637e+00,-2.021098e+00,-7.326831e-02
25%,-1.089076e+00,-9.031536e-01,-8.063453e-02,-6.606652e-01,-4.448281e-01,-5.535954e-01,-5.842379e-01,-6.924563e-01,-7.361236e-01,-7.326831e-02
50%,5.533922e-02,1.367805e-01,1.732292e-01,-4.020255e-02,4.691190e-01,-1.364774e-01,7.082076e-02,-1.403660e-01,-9.833712e-03,-7.326831e-02
75%,1.199754e+00,6.567476e-01,4.089598e-01,4.927389e-01,6.696628e-01,3.904086e-01,6.741643e-01,5.344111e-01,4.929823e-01,-7.326831e-02
max,1.199754e+00,1.696682e+00,1.007353e+00,2.819865e+00,1.261610e+00,1.033538e+01,2.484195e+00,3.417549e+00,3.007063e+00,2.157228e+01


# Tuning of hyperparameters: BatchSize and Epochs

In [9]:
from sklearn.model_selection import GridSearchCV,KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [10]:
def create_model():
    model=Sequential()
    model.add(Dense(15,input_dim=10,kernel_initializer='uniform',activation='relu'))
    model.add(Dense(20,kernel_initializer='uniform',activation='relu'))
    model.add(Dense(1,kernel_initializer='uniform',activation='sigmoid'))
    adam=Adam(lr=0.01)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [11]:
model=KerasClassifier(build_fn=create_model,verbose=0)
batch_size=[20,50,77]
epochs=[25,75,100]
param_grid=dict(batch_size=batch_size,epochs=epochs)
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=KFold(),verbose=10)
grid_result=grid.fit(x_standardized,y)

C:\Users\rahul\AppData\Local\Temp/ipykernel_15628/1538552576.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model,verbose=0)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=20, epochs=25....................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 1/5; 1/9] END .....batch_size=20, epochs=25;, score=0.000 total time=   2.1s
[CV 2/5; 1/9] START batch_size=20, epochs=25....................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

[CV 3/5; 9/9] END ....batch_size=77, epochs=100;, score=0.039 total time=   1.2s
[CV 4/5; 9/9] START batch_size=77, epochs=100...................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 4/5; 9/9] END ....batch_size=77, epochs=100;, score=0.000 total time=   1.5s
[CV 5/5; 9/9] START batch_size=77, epochs=100...................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 5/5; 9/9] END ....batch_size=77, epochs=100;, score=0.000 total time=   1.2s


# Tuning of hyperparameters: LearningRate and DropOut

In [12]:
from keras.layers import Dropout
def create_model(learning_rate,dropout_rate):
    model=Sequential()
    model.add(Dense(10,input_dim=10,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(6,input_dim=10,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation='sigmoid'))
    
    
    adam=Adam(lr=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
    return model

In [13]:
model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)
learning_rate=[0.1,0.01,0.001]
dropout_rate=[0.2,0.02,0.002]
param_grids=dict(learning_rate=learning_rate,dropout_rate=dropout_rate)
grid=GridSearchCV(estimator=model,param_grid=param_grids,cv=KFold(),verbose=10)

C:\Users\rahul\AppData\Local\Temp/ipykernel_15628/3106369480.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)


In [14]:
grid_result=grid.fit(x_standardized,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.2, learning_rate=0.1.........................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 1/5; 1/9] END dropout_rate=0.2, learning_rate=0.1;, score=0.000 total time=   1.1s
[CV 2/5; 1/9] START dropout_rate=0.2, learning_rate=0.1.........................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 2/5; 1/9] END dropout_rate=0.2, learning_rate=0.1;, score=0.000 total time=   1.0s
[CV 3/5; 1/9] START dropout_rate=0.2, learning_rate=0.1.........................


...

[CV 3/5; 9/9] END dropout_rate=0.002, learning_rate=0.001;, score=0.039 total time=   1.3s
[CV 4/5; 9/9] START dropout_rate=0.002, learning_rate=0.001.....................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 4/5; 9/9] END dropout_rate=0.002, learning_rate=0.001;, score=0.000 total time=   1.1s
[CV 5/5; 9/9] START dropout_rate=0.002, learning_rate=0.001.....................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 5/5; 9/9] END dropout_rate=0.002, learning_rate=0.001;, score=0.000 total time=   1.1s


In [15]:
print('Best : {}, Using {}'.format(grid_result.best_score_,grid_result.best_params_))
means=grid_result.cv_results_['mean_test_score']
stds=grid_result.cv_results_['std_test_score']
params=grid_result.cv_results_['params']
for mean,stdevs,param in zip (means,stds,params):
    print('{},{} with : {}'.format (mean,stdevs,param))

Best : 0.00776699036359787, Using {'dropout_rate': 0.2, 'learning_rate': 0.1}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.2, 'learning_rate': 0.1}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.02, 'learning_rate': 0.1}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.02, 'learning_rate': 0.01}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.02, 'learning_rate': 0.001}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.002, 'learning_rate': 0.1}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.002, 'learning_rate': 0.01}
0.00776699036359787,0.01553398072719574 with : {'dropout_rate': 0.002, 'learning_rate': 0.001}


# Activation function and Kernel initializer

In [17]:
def create_model(activation_function,init):
    model=Sequential()
    model.add(Dense(9,input_dim=10,kernel_initializer=init,activation=activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(5,input_dim=10,kernel_initializer=init,activation=activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid'))
    
    
    adam=Adam(lr=0.1)
    model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
    return model

In [18]:
model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)
activation_function=['softmax','relu','tanh','linear']
init=['uniform','normal','zero']
param_grid=dict(activation_function=activation_function,init=init)
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=KFold(),verbose=10)
grid_result=grid.fit(x_standardized,y)

C:\Users\rahul\AppData\Local\Temp/ipykernel_15628/4068080273.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)
C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 1/5; 1/12] END activation_function=softmax, init=uniform;, score=0.000 total time=   1.2s
[CV 2/5; 1/12] START activation_function=softmax, init=uniform..................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 2/5; 1/12] END activation_function=softmax, init=uniform;, score=0.000 total time=   1.2s
[CV 3/5; 1/12] START activation_function=softmax, init=uniform..................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

[CV 3/5; 12/12] END activation_function=linear, init=zero;, score=0.039 total time=   1.0s
[CV 4/5; 12/12] START activation_function=linear, init=zero.....................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 4/5; 12/12] END activation_function=linear, init=zero;, score=0.000 total time=   1.1s
[CV 5/5; 12/12] START activation_function=linear, init=zero.....................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 5/5; 12/12] END activation_function=linear, init=zero;, score=0.000 total time=   1.5s


# Number of neurons in activation layer

In [21]:
def create_model(neuron1,neuron2):
    model=Sequential()
    model.add(Dense(neuron1,input_dim=10,kernel_initializer='normal',activation='linear'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron2,input_dim=10,kernel_initializer='normal',activation='linear'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation='sigmoid'))
    
    
    adam=Adam(lr=0.1)
    model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])
    return model

In [22]:
model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)
neuron1=[30,20,10]
neuron2=[10,15,20]
param_grid=dict(neuron1=neuron1,neuron2=neuron2)
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=KFold(),verbose=10)
grid_result=grid.fit(x_standardized,y)

C:\Users\rahul\AppData\Local\Temp/ipykernel_15628/3726098652.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model,verbose=0,batch_size=20,epochs=25)
C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=30, neuron2=10......................................
[CV 1/5; 1/9] END .......neuron1=30, neuron2=10;, score=0.096 total time=   1.4s
[CV 2/5; 1/9] START neuron1=30, neuron2=10......................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 2/5; 1/9] END .......neuron1=30, neuron2=10;, score=0.077 total time=   1.2s
[CV 3/5; 1/9] START neuron1=30, neuron2=10......................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

[CV 3/5; 9/9] END .......neuron1=10, neuron2=20;, score=0.097 total time=   1.1s
[CV 4/5; 9/9] START neuron1=10, neuron2=20......................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 4/5; 9/9] END .......neuron1=10, neuron2=20;, score=0.117 total time=   1.1s
[CV 5/5; 9/9] START neuron1=10, neuron2=20......................................


C:\Users\rahul\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[CV 5/5; 9/9] END .......neuron1=10, neuron2=20;, score=0.155 total time=   1.0s


In [23]:
print('Best : {}, Using {}'.format(grid_result.best_score_,grid_result.best_params_))
means=grid_result.cv_results_['mean_test_score']
stds=grid_result.cv_results_['std_test_score']
params=grid_result.cv_results_['params']
for mean,stdevs,param in zip (means,stds,params):
    print('{},{} with : {}'.format (mean,stdevs,param))

Best : 0.10838312357664108, Using {'neuron1': 30, 'neuron2': 20}
0.10063480287790298,0.01482313155767947 with : {'neuron1': 30, 'neuron2': 10}
0.10255787968635559,0.011971599613186822 with : {'neuron1': 30, 'neuron2': 15}
0.10838312357664108,0.021012903977798435 with : {'neuron1': 30, 'neuron2': 20}
0.10647871643304825,0.02904168973238719 with : {'neuron1': 20, 'neuron2': 10}
0.10451829731464386,0.02008267579574321 with : {'neuron1': 20, 'neuron2': 15}
0.10646004676818847,0.023245493075346284 with : {'neuron1': 20, 'neuron2': 20}
0.10646004676818847,0.023245493075346284 with : {'neuron1': 10, 'neuron2': 10}
0.10646004676818847,0.023245493075346284 with : {'neuron1': 10, 'neuron2': 15}
0.10647871643304825,0.027739056477971244 with : {'neuron1': 10, 'neuron2': 20}


# Train model with optimum values of hyperparameter

In [24]:
model=Sequential()
model.add(Dense(30,input_dim=10,activation='linear',kernel_initializer='normal'))
model.add(Dropout(0.2))
model.add(Dense(10,input_dim=10,kernel_initializer='normal',activation='linear'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

adam=Adam(lr=0.1)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.fit(x,y, verbose=0, batch_size=20, epochs=25)

In [27]:
y_predict=model.predict(x_standardized)
cutoff = 0.7
y_predict_classes = np.zeros_like(y_predict)
y_predict_classes[y_predict > cutoff] = 1

In [28]:
y_classes = np.zeros_like(y_predict)
y_classes[y > cutoff] = 1

In [30]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_classes,y_predict_classes))

0.5183752417794971
